In [13]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt


import tensorflow as tf
    
   
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids


def load_embeddings(embeddings_file_path):
    """
    훈련된 텍스트 임베딩을 불러옴
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

def load_bounding_boxes(dataset_dir):
    """
    이미지와 그에 상응하는 바운딩 박스를 짝지어 딕셔너리로 만들어 출력
    """
    # 바운딩 박스 전체 경로
    bounding_boxes_path = os.path.join(dataset_dir, 'list_bbox_celeba_pure.csv')
    file_paths_path = os.path.join(dataset_dir, 'list_filenames.csv')

    # bounding_boxes.txt 와 images.txt 파일을 읽어옴
    df_bounding_boxes = pd.read_csv(bounding_boxes_path, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, header=None)

    # 전체 이미지 파일 명이 순서대로 적힌 리스트를 만듬
    file_names = df_file_names[0].tolist() 

    # 파일 이름에 대응하는 바운딩 박스가 들어갈 딕셔너리를 만듬 (딕셔너리는 크기를 임의로 증가시킬수 있으므로 초기 사이즈는 아무렇게나)
    filename_boundingbox_dict = {}

    # 이미지 파일과 그에 해당하는 바운딩 박스를 딕셔너리로 만듬 (key = 이미지 파일 이름)
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][:].tolist()
        key = file_names[i][:-4] + '.jpg'
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict
'''
새 이미지가 크롭핑 되어있지 않기 크롭하기 위한 바운딩 박스 좌표 값이 파일에 주어지며,
그 파일을 토대로 이미지를 크로핑 한 후,
크로핑된 모든 이미지를 지정한 이미지 크기 (image_size) 값으로 바꾼다
'''
def get_img(img_path, bbox, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        pass
    '''
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    '''
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img


def load_dataset(filenames_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    '''
    class_ids = load_class_ids(class_info_file_path)
    '''
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    # 각 이미지에 해당하는 바운딩 박스 딕셔너리를 추출하여 get_img 함수로 크로핑되고 같은 크기로 바뀐 이미지를 
    for index, filename in enumerate(filenames):
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{0}/images/{1}'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)
            '''
            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            '''
            embedding = all_embeddings[index, :]
            # X = 정제한 이미지 리스트
            X.append(np.array(img))
            '''
            # y = 정제한 이미지 인덱스
            y.append(class_ids[index])
            '''
            # embeddings = 
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, embeddings


def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c


def build_ca_model():
    """
    (1024,)의 텍스트 인코더 신경망의 출력을 입력으로 받고 (256,) 의 텐서를 출력
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_embedding_compressor_model():
    """
    입력 속성값 (40,) 을 (128,) 의 벡터로 확장하는 네트워크
    """
    input_layer = Input(shape=(40,))
    x = Dense(10)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_stage1_generator():
    """
    Stage-I 의 generator 
    *** 이 신경망 안에 CA 신경망과 생성기 신경망이 들어가 있다!!!! ***
    그러므로, 입력으로 텍스트 임베딩 출력 (1024,)과 잡음 변수(100,) 을 받는다
    """
    '''
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))
    gen_input = Concatenate(axis=1)([c, input_layer2])
    '''
    # 텍스트 조건부 변수를 잡음 변수와 접합(concatenation) -> cGAN

    input_layer = Input(shape=(40,))
    x = Dense(128 * 8 * 4 * 4, use_bias=False)(input_layer)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=input_layer, outputs=x)
    '''
    stage - I gen 은 입력된 문장의 임베딩을 바탕으로 (+잡음 변수) 이미지를 생성 함 
    '''
    return stage1_gen


def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    '''
    실제 이미지에 해당하는 압축된 임베딩을 입력
    '''
    input_layer2 = Input(shape=(4, 4, 40))

    '''
    입력 이미지와 압축 텍스트 임베딩을 합침
    '''
    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=x2)
    '''
    출력은 입력 이미지가 진짜인지 가짜인지에 관한 확률(sigmoid)을 출력
    '''
    return stage1_dis


def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(40,))
    input_layer3 = Input(shape=(4, 4, 40))

    x = gen_model(input_layer)

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer3], outputs=valid)
    return model


def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss


def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)


def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()


def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    with callback.as_default():
          tf.summary.scalar(name, loss, batch_no)
          callback.flush()








'''
Stage - I stackGAN 훈련
'''
if __name__ == '__main__':
    '''
    하이퍼파라미터(불변 파라미터) 지정
    '''
    data_dir = "/home/csle/Desktop/CelebA_dataset_reduce"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    image_size = 64
    batch_size = 128
    z_dim = 10
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = 1500
    condition_dim = 40

    embeddings_file_path_train = train_dir + "/attr_(embeddings).pickle"
    embeddings_file_path_test = test_dir + "/attr_(embeddings).pickle"

    filenames_file_path_train = train_dir + "/filenames.pickle"
    filenames_file_path_test = test_dir + "/filenames.pickle"

    '''
    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"
    '''

    cub_dataset_dir = data_dir + "/img_align_celeba"

    '''
    optimizer 정의
    '''
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    """"
    dataset 로드하기
    """
    X_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                      cub_dataset_dir=cub_dataset_dir,
                                                      embeddings_file_path=embeddings_file_path_train,
                                                      image_size=(64, 64))

    X_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                   cub_dataset_dir=cub_dataset_dir,
                                                   embeddings_file_path=embeddings_file_path_test,
                                                   image_size=(64, 64))
    
    print('finish')


embeddings:  (16000, 40)
Embeddings shape: (16000, 40)
embeddings:  (4000, 40)
Embeddings shape: (4000, 40)
finish


In [15]:
#with tf.device('/gpu:0'): 

import tensorflow as tf 

config = tf.compat.v1.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.6 
session = tf.compat.v1.Session(config=config)

    
"""
신경망 빌드 & compile
"""
'''
ca_model = build_ca_model()
ca_model.compile(loss="binary_crossentropy", optimizer="adam")
'''
stage1_dis = build_stage1_discriminator()
stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

stage1_gen = build_stage1_generator()
stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

embedding_compressor_model = build_embedding_compressor_model()
embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

'''
stage-I GAN 빌드& 컴파일
이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
'''
adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer=gen_optimizer, metrics=None)

'''
tensorboard.set_model(stage1_gen)
tensorboard.set_model(stage1_dis)
tensorboard.set_model(ca_model)
tensorboard.set_model(embedding_compressor_model)
'''

# Generate an array containing real and fake values
# Apply label smoothing as well
real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1
'''
매 epoch 마다 아래를 반복함
'''
for epoch in range(epochs):
    print("========================================")
    print("Epoch is:", epoch)
    print("Number of batches", int(X_train.shape[0] / batch_size))

    gen_losses = []
    dis_losses = []

    # Load data and train model
    number_of_batches = int(X_train.shape[0] / batch_size)
    for index in range(number_of_batches):
        print("Batch:{}".format(index+1))
        '''
        모델에 입력으로 들어갈 이미지와 텍스트 임베딩을 받아옴 (각 텍스트 임베딩은 각 이미지에 대응 됨)
        '''
        # 원래는 CA 의 출력이 stage-I 으로 들어가야 하지만 gen 안에 CA 가 있음
        z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        # 배치 사이즈만큼 훈련(실제) 이미지를 추출
        image_batch = X_train[index * batch_size:(index + 1) * batch_size]
        # 추출한 이미지에 대응하는 임베딩을 추출
        embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
        # 이미지들을 정규화하여 값을 작게 만듬
        image_batch = (image_batch - 127.5) / 127.5

        # stage-I 의 gen 에서 텍스트 임베딩을 바탕으로 저 해상도 fake 이미지를 생성
        # 이때, 두 stage 는 랜덤하게 생성한 텍스트 임베딩을 나머지 입력으로 받음
        fake_images = stage1_gen.predict(embedding_batch, verbose=3)

        # stage-I dis 에 들어갈 압축 텍스트 임베딩을 랜덤하게 생성한 텍스트 임베딩 기반으로 생성
        '''
        compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
        '''
        compressed_embedding = np.reshape(embedding_batch, (-1, 1, 1, condition_dim))
        compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))
        '''
        discriminator 에서 입력 이미지가 CNN 을 통과한 결과와 속성값 (40,) 을 concatenate 해 주기 위해서는,
        속성값 (40,) 를 compressor 네트워크를 통해 (128,) 로 확장하고,
        reshape, tile 함수를 이용해 CNN 결과 (4, 4, 512) 와 (4, 4, :) 부분을 맞추어 주어야 한다.
        '''

        """
        stage-I dis 를 훈련함
        """
        # 실제 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 1 (이미지가 진짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                  np.reshape(real_labels, (batch_size, 1)))
        # gen 이 생성한 가짜 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 가짜 이미지를 잘 분류 하게끔 훈련
        dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                  np.reshape(fake_labels, (batch_size, 1)))
        # 실제 이미지와 압축 텍스트 임베딩을 '어긋나게' 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                   np.reshape(fake_labels[1:], (batch_size-1, 1)))
        
        d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))
        
        print("d_loss_real:{}".format(dis_loss_real))
        print("d_loss_fake:{}".format(dis_loss_fake))
        print("d_loss_wrong:{}".format(dis_loss_wrong))

        print("d_loss:{}".format(d_loss))

        """
        stage-I GAN 을 훈련함
        이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
        """
        g_loss = adversarial_model.train_on_batch([embedding_batch, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
        print("g_loss:{}".format(g_loss))

        dis_losses.append(d_loss)
        gen_losses.append(g_loss)

    """
    각 epoch 마다 Tensorboard 에 loss 의 전체 배치 평균값 저장
    """
    writer = tf.summary.create_file_writer("logs/".format(time.time()))
    write_log(writer, 'discriminator_loss', np.mean(dis_losses), epoch)
    write_log(writer, 'generator_loss', np.mean(gen_losses[0]), epoch)
    '''
    tf.summary.scalar('discriminator_loss', np.mean(dis_losses), epoch)
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]), epoch)

    tf.summary.scalar('discriminator_loss', np.mean(dis_losses))
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]))
    summary_op = tf.summary.merge()
    summary_writer = tf.summary.FileWriter("logs/".format(time.time()))
    '''    
    # 매 두번의 epoch 마다 이미지 gen & 이미지 저장
    if epoch % 2 == 0:
        # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
        z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
        embedding_batch = embeddings_test[0:batch_size]
        fake_images = stage1_gen.predict_on_batch(embedding_batch)

        # Save images
        for i, img in enumerate(fake_images[:10]):
            save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))

    if epoch % 50 == 0:
        # Save models
        stage1_gen.save_weights("weights/stage1_gen_epoch_{}.h5".format(epoch))
        stage1_dis.save_weights("weights/stage1_dis_epoch_{}.h5".format(epoch))

'''
이제 훈련된 stage-I 의 generator 와 discriminator 을 얻음 (+ embedding_compressor) 
'''

Epoch is: 0
Number of batches 125
Batch:1
d_loss_real:0.6371667385101318
d_loss_fake:3.2823572158813477
d_loss_wrong:9.506380081176758
d_loss:3.5157676935195923
g_loss:0.7067953944206238
Batch:2
d_loss_real:0.5911878347396851
d_loss_fake:4.769085884094238
d_loss_wrong:0.07942267507314682
d_loss:1.5077210571616888
g_loss:0.819270133972168
Batch:3
d_loss_real:4.523982048034668
d_loss_fake:0.024801677092909813
d_loss_wrong:1.7190333604812622
d_loss:2.697949783410877
g_loss:0.7329603433609009
Batch:4
d_loss_real:1.776766061782837
d_loss_fake:1.9808142185211182
d_loss_wrong:1.0165823698043823
d_loss:1.6377321779727936
g_loss:0.9153938293457031
Batch:5
d_loss_real:2.779651403427124
d_loss_fake:0.0008177928393706679
d_loss_wrong:1.046659231185913
d_loss:1.651694957719883
g_loss:0.8049939870834351
Batch:6
d_loss_real:1.796554684638977
d_loss_fake:0.06509445607662201
d_loss_wrong:1.2460230588912964
d_loss:1.2260567210614681
g_loss:0.7480756640434265
Batch:7
d_loss_real:1.6343605518341064
d_loss

g_loss:0.32560110092163086
Batch:49
d_loss_real:1.0005184412002563
d_loss_fake:0.0020281560719013214
d_loss_wrong:0.8045695424079895
d_loss:0.7019086452201009
g_loss:0.3251962661743164
Batch:50
d_loss_real:0.94480299949646
d_loss_fake:0.0018993898993358016
d_loss_wrong:0.8215463161468506
d_loss:0.6782629262597766
g_loss:0.32533878087997437
Batch:51
d_loss_real:0.9909409284591675
d_loss_fake:0.0018248187843710184
d_loss_wrong:0.7394067049026489
d_loss:0.6807783451513387
g_loss:0.3255774676799774
Batch:52
d_loss_real:0.993722677230835
d_loss_fake:0.001562396064400673
d_loss_wrong:0.759868323802948
d_loss:0.6872190185822546
g_loss:0.32573914527893066
Batch:53
d_loss_real:0.956173837184906
d_loss_fake:0.0015327618457376957
d_loss_wrong:0.8081673383712769
d_loss:0.6805119436467066
g_loss:0.3273189067840576
Batch:54
d_loss_real:0.9703671932220459
d_loss_fake:0.0015663416124880314
d_loss_wrong:0.7501419186592102
d_loss:0.6731106616789475
g_loss:0.3282974660396576
Batch:55
d_loss_real:0.975103

g_loss:0.32752183079719543
Batch:101
d_loss_real:0.9045873284339905
d_loss_fake:0.0017567310715094209
d_loss_wrong:0.72516268491745
d_loss:0.6340235182142351
g_loss:0.32699862122535706
Batch:102
d_loss_real:0.9035064578056335
d_loss_fake:0.0007519443752244115
d_loss_wrong:0.713912844657898
d_loss:0.6304194261610974
g_loss:0.32604700326919556
Batch:103
d_loss_real:0.9051613807678223
d_loss_fake:0.0013534483732655644
d_loss_wrong:0.7638850212097168
d_loss:0.6438903077796567
g_loss:0.32689613103866577
Batch:104
d_loss_real:0.8960452079772949
d_loss_fake:0.0009372375207021832
d_loss_wrong:0.677884578704834
d_loss:0.6177280580450315
g_loss:0.32785430550575256
Batch:105
d_loss_real:0.9096627831459045
d_loss_fake:0.001093599246814847
d_loss_wrong:0.6995508670806885
d_loss:0.6299925081548281
g_loss:0.32614341378211975
Batch:106
d_loss_real:0.8946870565414429
d_loss_fake:0.001110250479541719
d_loss_wrong:0.6822888255119324
d_loss:0.61819329726859
g_loss:0.3255929946899414
Batch:107
d_loss_real:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


g_loss:0.3254886269569397


...

d_loss_real:0.32560670375823975
d_loss_fake:6.955287972232327e-06
d_loss_wrong:0.000533139449544251
d_loss:0.162938375563499
g_loss:0.7092936038970947
Batch:80
d_loss_real:0.34469160437583923
d_loss_fake:1.4071354598854668e-05
d_loss_wrong:0.0011016891803592443
d_loss:0.17262474232165914
g_loss:0.4338333010673523
Batch:81
d_loss_real:0.37603893876075745
d_loss_fake:1.3228337593318429e-05
d_loss_wrong:0.0006059756851755083
d_loss:0.18817427038607093
g_loss:0.6248270273208618
Batch:82
d_loss_real:0.3261813223361969
d_loss_fake:7.434533472405747e-06
d_loss_wrong:0.0002140105061698705
d_loss:0.16314602242800902
g_loss:0.6758147478103638
Batch:83
d_loss_real:0.3410666882991791
d_loss_fake:1.3696007954422385e-05
d_loss_wrong:0.0006387245375663042
d_loss:0.17069644928596972
g_loss:0.5136041045188904
Batch:84
d_loss_real:0.3387486934661865
d_loss_fake:1.2540574971353635e-05
d_loss_wrong:0.0015036541735753417
d_loss:0.16975339542022994
g_loss:0.6650639772415161
Batch:85
d_loss_real:0.3278609514

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

Epoch is: 105
Number of batches 125
Batch:1
d_loss_real:0.33378422260284424
d_loss_fake:2.289901749463752e-05
d_loss_wrong:0.0007116260821931064
d_loss:0.16707574257634406
g_loss:0.48483628034591675
Batch:2
d_loss_real:0.33272528648376465
d_loss_fake:3.690100129460916e-05
d_loss_wrong:0.0006475363625213504
d_loss:0.16653375258283631
g_loss:0.5579661726951599
Batch:3
d_loss_real:0.33032360672950745
d_loss_fake:3.471402305876836e-05
d_loss_wrong:0.005142477340996265
d_loss:0.16645610120576748
g_loss:0.5179750919342041
Batch:4
d_loss_real:0.3351442813873291
d_loss_fake:2.0567526007653214e-05
d_loss_wrong:0.0007037075702100992
d_loss:0.167753209467719
g_loss:0.6194518208503723
Batch:5
d_loss_real:0.32876211404800415
d_loss_fake:1.6554420653847046e-05
d_loss_wrong:0.00019485419034026563
d_loss:0.1644339091767506
g_loss:0.5872417688369751
Batch:6
d_loss_real:0.33776167035102844
d_loss_fake:2.7483270969241858e-05
d_loss_wrong:0.00042968199704773724
d_loss:0.16899512649251847
g_loss:0.46292749

g_loss:0.5552612543106079
Batch:52
d_loss_real:0.32795730233192444
d_loss_fake:1.255952247447567e-05
d_loss_wrong:0.0010657869279384613
d_loss:0.16424823777856545
g_loss:0.608098030090332
Batch:53
d_loss_real:0.32770898938179016
d_loss_fake:1.3251744348963257e-05
d_loss_wrong:0.0007465754169970751
d_loss:0.1640444514812316
g_loss:0.633855402469635
Batch:54
d_loss_real:0.3284100592136383
d_loss_fake:8.060994332481641e-06
d_loss_wrong:0.0008042874978855252
d_loss:0.16440811672987365
g_loss:0.6006412506103516
Batch:55
d_loss_real:0.32822757959365845
d_loss_fake:1.527752283436712e-05
d_loss_wrong:0.0016946644755080342
d_loss:0.16454127529641482
g_loss:0.6315898299217224
Batch:56
d_loss_real:0.3335966467857361
d_loss_fake:2.328209302504547e-05
d_loss_wrong:0.0028876380529254675
d_loss:0.16752605342935567
g_loss:0.49672091007232666
Batch:57
d_loss_real:0.3446630537509918
d_loss_fake:1.148042701970553e-05
d_loss_wrong:0.0005462697008624673
d_loss:0.17247096440746645
g_loss:0.5980696678161621


KeyboardInterrupt: 